# A .PY Chatbot 
# (Integrated into a Facebook Messenger bot using ngrok and Flask)

### Introduction
The notebook consists of various sections that explains how a bot can be developed from scratch. Each of the sections explained in detail later on, are code snippets that need to be incorporated in a bot to make it complete.

The next steps would be to make it more exciting by letting the bot interact with people and improve the code in an incremental manner as and when new real world scenarios are observed. 
This is done by integrating the below sections of python code with facebook messenger bot where it the bot would respond to people messaging a particular page on Facebook.

## 1. Chatbots basics
### 1.1. Receiving a message and generating a response

 This section explains how a bot would read a message and acknowledge the message by generating a response.
 As of now, the response is not a valid response but just to acknowledge that it received a message

In [4]:
bot_template = "BOT : {0}"
user_template = "USER : {0}"

def respond(message):
    # Concatenate the user's message to the end of a standard bot respone
    bot_message = "I can hear you! You said: " + message
    # Return the result
    return bot_message

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))
    
send_message("Hello")

USER : Hello
BOT : I can hear you! You said: Hello


### 1.2. Generating a valid response

In this section, the bot generates a valid response to a set of questions using a simple dictionary approach in Python.

The set of questions needs to be entered in the dictionary exactly the way it is going to be asked by the person interacting with the bot. The answers also need to be entered in the dictonary as a key value pair with each of the question.

Every time the bot is asked the question "what's your name?", the response would be "my name is Greg".

In [9]:
# Define variables
name = "Greg"
weather = "sunny"

# Define a dictionary with the predefined responses
responses = {
  "what's your name?": "my name is {0}".format(name),
  "what's today's weather?": "the weather is {0}".format(weather),
  "default": "default message"
}

# Return the matching response if there is one, default otherwise
def respond(message):
    # Check if the message is in the responses
    if message in responses:
        # Return the matching message
        bot_message = responses[message]
    else:
        # Return the "default" message
        bot_message = responses["default"]
    return bot_message

send_message("what's your name?")
send_message("what's today's weather?")

USER : what's your name?
BOT : my name is Greg
USER : what's today's weather?
BOT : the weather is sunny


### 1.3. Addding variations to the response

The same response "My name is Greg" would be boring to read after a while and hence we need to add variation and randomness to response. This variation also adds to the personality of the bot and would make the conversation much more interesting for the user interacting with the bot.

This can be done by including multiple response formats as values in the same dictionary.


In [10]:
# Import the random module
import random

# Define a dictionary containing a list of responses for each message
responses = {
  "what's your name?": [
      "my name is {0}".format(name),
      "they call me {0}".format(name),
      "I go by {0}".format(name)
   ],
  "what's today's weather?": [
      "the weather is {0}".format(weather),
      "it's {0} today".format(weather)
    ],
  "default": ["I don't have an answer","Only Einstein can answer this", "You're smart! Stop asking me such difficult questions"]
}

# Use random.choice() to choose a matching response
def respond(message):
    # Check if the message is in the responses
    if message in responses:
        # Return a random matching response
        bot_message = random.choice(responses[message])
    else:
        # Return a random "default" response
        bot_message = random.choice(responses["default"])
    return bot_message

send_message("what's your name?")

USER : what's your name?
BOT : I go by Greg


### 1.4. Understanding intent using punctuation

The most basic way of understanding the intent of a message written to a bot would be by having a look at the punctuation.
A basic bot handling a set of 3-5 questions can be easily set up by incorporating undersating intent along with the sections above.

The bot can generate witty responses as shown below just by checking the intent of the message. This bots would generate replies such as "That's a difficult one. Ask me something else", "That's a good question. Let me think" for any question outside the domain of it's understanding.

In [14]:
import random

responses = {
  "question": [
      "That's a difficult one. Ask me something else",
      "That's a good question. Let me think",
      "I don't have an answer for that question"
   ],
  "statement": ["WOW! That's great",
    "Okay! Let's start with asking a question now?",
    "Okay! How may I help you?"
    ]}

def respond(message):
    # Check for a question mark
    if message.endswith("?"):
        # Return a random question
        return random.choice(responses["question"])
    # Return a random statement
    return random.choice(responses["statement"])


# Send messages ending in a question mark
send_message("what's today's weather?")
send_message("what's today's weather?")

# Send messages which don't end with a question mark
send_message("I love building chatbots")
send_message("I love building chatbots")


USER : what's today's weather?
BOT : I don't have an answer for that question
USER : what's today's weather?
BOT : I don't have an answer for that question
USER : I love building chatbots
BOT : Okay! How may I help you?
USER : I love building chatbots
BOT : WOW! That's great


### 1.5. Text munging with regular expressions

The bot developed above would not generate a response for a question that does not match the ones in the dictionary. 

Hence, regular expressions needs to be incorporated to make the bot better at understanding a set of questions that can be asked. This along with a replacement of simple pronouns makes the bot respond in interesting ways to a huge set of questions as shown below.

In [13]:
import re

rules = {'do you remember (.*)': ['Did you think I would forget {0}', "Why haven't you been able to forget {0}", 'What about {0}', 'Yes .. and?'], 
         'do you think (.*)': ['if {0}? Absolutely.', 'No chance'], 
         'if (.*)': ["Do you really think it's likely that {0}", 'Do you wish that {0}', 'What do you think about {0}', 'Really--if {0}'],
         'I want (.*)': ['What would it mean if you got {0}', 'Why do you want {0}', "What's stopping you from getting {0}"]}

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern, responses in rules.items():
        # Create a match object
        match = re.search(pattern,message)
        if match is not None:
            # Choose a random response
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

def replace_pronouns(message):
    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me','you',message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my','your',message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your','my',message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you','me',message)

    return message

def respond(message):
    # Call match_rule
    response, phrase = match_rule(rules,message)
    if '{0}' in response:
        # Replace the pronouns in the phrase
        phrase = replace_pronouns(phrase)
        # Include the phrase in the response
        response = response.format(phrase)
    return response

respond("I want an iphone")

'What would it mean if you got an iphone'

## 2. Understanding Natural Language
### 2.1. Understanding intent using keywords

This section involves understanding intent of a message using keywords that explain a particular intent.

In simple words, if the words 'hello', 'hi' or 'hey' are present in the message, the intent is to greet the bot and hence a relevant response needs to be generated. So, instead of looking for the entire question in the dictionary, we try and identify the intent of the message and then generate a response for the message.

In [15]:
keywords={'goodbye': ['bye', 'farewell'], 'thankyou': ['thank', 'thx'], 'greet': ['hello', 'hi', 'hey']}
responses={'default': 'I dont understand', 'goodbye': 'goodbye for now','greet': 'Hello you! :)', 'thankyou': 'you are very welcome'}

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = '|'.join(keys)
    
# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if re.search(pattern,message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

# Send messages
send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

USER : hello!
BOT : Hello you! :)
USER : bye byeee
BOT : goodbye for now
USER : thanks very much!
BOT : you are very welcome


### 2.2. Extracting the 'name' entity using regular expressions

This section involves extracting the 'name' entity from the message.
It is definitely not the most efficient way of extracting the 'name' entity but regular expressions can be used to extract it as shown below.

In [21]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile(r"\b(name|call)")
    
    # Create a pattern for finding capitalized words
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

greet_response = ("Hello", "Hi", "Greetings", "Sup", "What's up",)

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi! Do you have a question for me?"
    else:
        return random.choice(greet_response)+", {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Alex")
send_message("people call me Cassandra")
send_message("hi, i have no name")

USER : my name is David Copperfield
BOT : Greetings, David Copperfield!
USER : call me Alex
BOT : Hi, Alex!
USER : people call me Cassandra
BOT : Sup, Cassandra!
USER : hi, i have no name
BOT : Hi! Do you have a question for me?


In [119]:
sentences = [' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
 ' what flights are available from pittsburgh to baltimore on thursday morning',
 ' what is the arrival time in san francisco for the 755 am flight leaving washington',
 ' cheapest airfare from tacoma to orlando',
 ' round trip fares from pittsburgh to philadelphia under 1000 dollars',
 ' i need a flight tomorrow from columbus to minneapolis',
 ' what kind of aircraft is used on a flight from cleveland to dallas',
 ' show me the flights from pittsburgh to los angeles on thursday',
 ' all flights from boston to washington',
 ' what kind of ground transportation is available in denver',
 ' show me the flights from dallas to san francisco',
 ' show me the flights from san diego to newark by way of houston',
 ' what is the cheapest flight from boston to bwi',
 ' all flights to baltimore after 6 pm',
 ' show me the first class fares from boston to denver',
 ' show me the ground transportation in denver',
 ' all flights from denver to pittsburgh leaving after 6 pm and before 7 pm',
 ' i need information on flights for tuesday leaving baltimore for dallas dallas to boston and boston to baltimore',
 ' please give me the flights from boston to pittsburgh on thursday of next week',
 ' i would like to fly from denver to pittsburgh on united airlines',
 ' show me the flights from san diego to newark',
 ' please list all first class flights on united from denver to baltimore',
 ' what kinds of planes are used by american airlines',
 " i'd like to have some information on a ticket from denver to pittsburgh and atlanta",
 " i'd like to book a flight from atlanta to denver",
 ' which airline serves denver pittsburgh and atlanta',
 " show me all flights from boston to pittsburgh on wednesday of next week which leave boston after 2 o'clock pm",
 ' atlanta ground transportation',
 ' i also need service from dallas to boston arriving by noon',
 ' show me the cheapest round trip fare from baltimore to dallas']

labels = ['atis_flight',
 'atis_flight',
 'atis_flight_time',
 'atis_airfare',
 'atis_airfare',
 'atis_flight',
 'atis_aircraft',
 'atis_flight',
 'atis_flight',
 'atis_ground_service',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_airfare',
 'atis_ground_service',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_aircraft',
 'atis_airfare',
 'atis_flight',
 'atis_airline',
 'atis_flight',
 'atis_ground_service',
 'atis_flight',
 'atis_airfare']

In [ ]:
import spacy 

# Load the spacy model: nlp
nlp = spacy.load('en')

# Calculate the length of sentences
n_sentences = len(sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length

# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector

### 2.3. Understanding intent using spacy and machine learning

#### 2.3.1. Using word vectors in spacy and cosine similarity

In this section, we initially find the word vector for each of the training samples that are already labeled.
Then, using cosine similarity the label of the test sample is approximated by comparing the word vector value of the test with each of the training sample word vector.

In [ ]:
## NEAREST NEIGHBOR or cosine similarity

# sentences_train is the sentences to be used for training the classifier
sentences_train = ["i want to fly from boston at 838 am and arrive in denver at 1110 in the morning",
  "what flights are available from pittsburgh to baltimore on thursday morning"]
labels_train = ["atis_flight","atis_flight" ]

import numpy as np
X_train_shape = (len(sentences_train),nlp.vocab.vectors_length)

X_train = np.zeros(X_train_shape)

for sentence in sentences_train:
    X_train[i,:] = nlp(sentence).vector

from sklearn.metrics.pairwise import cosine_similarity

test_message = """
i would like to find a flight from charlotte
to las vegas that makes a stop in st. louis"""

test_x = nlp(test_message).vector

scores = [cosine_similarity(X[i,:], test_x) for i in range(len(sentences_train)]
                                                           
labels_train[np.argsmax(scores)]                                            

'atis_flight'

#### 2.3.2. Using word vectors in spacy and SVM

In this section, the same test set consisting of word vectors are used to train the classifier but the classifier under consideration is SVM which is imported from sklearn.

In [ ]:
# Import SVC
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC()

# Fit the classifier using the training data
clf.fit(X_train,y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
n_correct = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        n_correct += 1

print("Predicted {0} correctly out of {1} test examples".format(n_correct, len(y_test)))

Predicted 162 correctly out of 201 test examples

### 2.4. Extracting entities using Spacy

In this section, we'll use spaCy's built-in entity recognizer to extract names, dates, and organizations from search queries.

The built in entity recognizer has a huge database consisting of names, organization names and dates it looks up from before classifying an entity.

It is a good built-in entity recognizer that can be directly used for bots but it is not very accurate.

In [ ]:
# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))

{'ORG': None, 'PERSON': None, 'DATE': '2010'}

{'ORG': None, 'PERSON': None, 'DATE': None}

#### 2.4.1 Assigning roles to entities using Spacys parser 

In this section, using spacy's entity recognizer we assign roles to entities.

This is done in 2 steps:

1. Identify the __'color'__ in the statement using the entitiy recognizer.
2. We then try to link it to the object it is associated with using the ancestor of the word of entity type __'item'.__

In [ ]:
# Create the document
doc = nlp("let's see that jacket in red and some blue jeans")

# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in word.ancestors:
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent
            item =  find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc) 

item: jacket has color : red

item: jeans has color : blue

### 2.5. Natural Language Understanding with RASA

RASA is a very popular library in python used for intent & entity recognition. The library is based on __spacy, scikit-learn & other libraries.__

RASA takes JSON files as input and the trainer then interprets the data using a pipeline namely 'spacy_sklearn' used below.
The pipeline 'spacy_sklearn' consists of the following tasks:

1. nlp_spacy (Spacys 'eng' nlp interpreter)
2. ner_crf (named entity recognition - conditional random field(ML) model for extracting entities)
3. ner_synonyms(maps entities with same meaning to same keys. eg: NYC, New York City)
4. intent_featurizer_spacy (Creates vector representation of sentences)
5. intent_classifier_sklearn (SVC classifier)

The output of the interpreter is a JSON file consisting of intents and entities shown in the output section below.

In [136]:
# Import necessary modules
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer

# Create args dictionary
args = {"pipeline":"spacy_sklearn"}

# Create a configuration and trainer
config = RasaNLUConfig(cmdline_args=args)
trainer = Trainer(config)

# Load the training data
training_data = load_data("./training_data.json")

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

# Try it out
print(interpreter.parse("I'm looking for a Mexican restaurant in the North of town"))

{'entities': [{'value': 'mexican', 'start': 18, 'end': 25, 'entity': 'cuisine', 'extractor': 'ner_crf'}, 
              {'value': 'north', 'start': 44, 'end': 49, 'entity': 'location', 'extractor': 'ner_crf'}], 
 'intent': {'confidence': 0.650310305546843, 'name': 'restaurant_search'}, 'text': "I'm looking for a Mexican restaurant in the North of town", 
 'intent_ranking': [{'confidence': 0.650310305546843, 'name': 'restaurant_search'}, {'confidence': 0.14214642992469767, 'name': 'goodbye'}, {'confidence': 0.1050839200738026, 'name': 'greet'}, {'confidence': 0.10245934445465688, 'name': 'affirm'}]}

#### 2.5.1. Entity recognition using context specific entity recognizer

The accuracy of the entity recognition module can be improved by using a context specific __ner_crf__. In the case of bots made for recommending restaurants to people, the MITIE entity recogniser is a NER that performs exceedingly well.

This can be done by customizing the pipeline used to interpret the data. An example of customizing the pipeline is shown below:

In [ ]:
# Import necessary modules
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer

pipeline = [
    "nlp_spacy",
    "tokenizer_spacy",
    "ner_crf"
]

# Create a config that uses this pipeline
config = RasaNLUConfig(cmdline_args={'pipeline':pipeline})

# Create a trainer that uses this config
trainer = Trainer(config)

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

# Parse some messages
print(interpreter.parse("show me Chinese food in the centre of town"))
print(interpreter.parse("I want an Indian restaurant in the west"))
print(interpreter.parse("are there any good pizza places in the center?"))

{'intent': {'confidence': 0.0, 'name': ''}, 'entities': [{'end': 34, 'entity': 'location', 'value': 'centre', 'extractor': 'ner_crf', 'start': 28}], 'text': 'show me Chinese food in the centre of town'}
    {'intent': {'confidence': 0.0, 'name': ''}, 'entities': [{'end': 16, 'entity': 'cuisine', 'value': 'indian', 'extractor': 'ner_crf', 'start': 10}, {'end': 39, 'entity': 'location', 'value': 'west', 'extractor': 'ner_crf', 'start': 35}], 'text': 'I want an Indian restaurant in the west'}
    {'intent': {'confidence': 0.0, 'name': ''}, 'entities': [{'end': 45, 'entity': 'location', 'value': 'center', 'extractor': 'ner_crf', 'start': 39}], 'text': 'are there any good pizza places in the center?'}

### 2.6. Accessing data from SQL database

The section below discusses building a chatbot(virtual assistant) for tasks like scheduling a meeting, booking a flight etc.

In such a case, the bot needs to access data from a SQL database to extract information and display it for the user. The code snippet below can be used to excecute a basic SELECT statement from the database.

In [ ]:
# Import sqlite3
import sqlite3

# Open connection to DB
conn = sqlite3.connect('hotels.db')

# Create a cursor
c = conn.cursor()

# Define area and price
area, price = "south", "hi"
t = (area, price)

# Execute the query
c.execute('SELECT * FROM hotels WHERE area=? AND price=?', t)

# Print the results
print(c.fetchall())

[('Grand Hotel', 'hi', 'south', 5)]

In [3]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect("hotels.db")
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query,t)
    # Return the results
    results = c.fetchall()
    return results

# Create the dictionary of column names and values
params = {"area":"south","price":"lo"}

# Find the hotels that match the parameters
print(find_hotels(params))

[('Cozy Cottage', 'lo', 'south', 2)]

### 2.7. Exploring a DB using Natural Language

This is an interesting section where the bot understands natural language and responds to the question based on the data present in the database.

This section involves identfying entities from the statement, querying the database and then generating a response based on the query output.

In [ ]:
responses = ["I'm sorry :( I couldn't find anything like that",
 '{} is a great hotel!',
 '{} or {} would work!',
 '{} is one option, but I know others too :)']

# Define respond()
def respond(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find hotels that match the dictionary
    results = find_hotels(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)
    
print(respond("I want an expensive hotel in the south of town"))

'Grand Hotel is a great hotel!'

#### 2.7.1. Adding memory to a bot

In this section, we discuss how to add memory to a bot in case the user wants to add more filters to the data retrieved in consequent messages.

A good example for this scenario would be a user looking for an expensive hotel in the first message and then narrowing down his search to a hotel in the north of town.

In [ ]:
# Define a respond function, taking the message and existing params as input
def respond(message, params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

# Initialize params dictionary
params = {}

# Pass the messages to the bot
for message in ["I want an expensive hotel", "in the north of town"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))

USER: I want an expensive hotel

BOT: Grand Hotel is one option, but I know others too :)

USER: in the north of town

BOT: Ben's BnB is a great hotel!

#### 2.7.2. Handling negation

In this section, we learn to handle neagation in a statement to identify the right filters to be applied while querying the database.

A simple approach is used below where we search for the keywords __not__ and __n't__ as a prefix of the entity.


In [11]:
tests = [("no I don't want to be in the south", {'south': False}),
 ('no it should be in the south', {'south': True}),
 ('no in the south not the north', {'north': False, 'south': True}),
 ('not north', {'north': False})]

# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([ phrase.index(e) + len(e) for e in ents])
    
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
        
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity is preceeded by a negation, give it the key False
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(negated_ents(test[0]) == test[1])

True
True
True
True


In [ ]:
# Define the respond function
def respond(message, params, neg_params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    ent_vals = [e["value"] for e in entities]
    # Look for negated entities
    negated = negated_ents(message, ent_vals)
    for ent in entities:
        if ent["value"] in negated and negated[ent["value"]]:
            neg_params[ent["entity"]] = str(ent["value"])
        else:
            params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_hotels(params, neg_params)
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Return the correct response
    return responses[n].format(*names), params, neg_params

# Initialize params and neg_params
params = {}
neg_params = {}

# Pass the messages to the bot
for message in ["I want a cheap hotel", "but not in the north of town"]:
    print("USER: {}".format(message))
    response, params, neg_params = respond(message, params, neg_params)
    print("BOT: {}".format(response))

### OUTPUT
USER: I want a cheap hotel
BOT: Cozy Cottage is a great hotel!
USER: but not in the north of town
BOT: I'm sorry :( I couldn't find anything like that

## 3. Dialogue with a Bot (Statefulness)

Until the now, the focus has been on mapping the entities and the intent of the message, and then generating a valid response.

Now, let's focus on adding some statefulness to the bot.

This means shifting between states based on the messages sent to the bot from the user. For simplicity, let's design a bot that will help you buy coffee and have three different states for this bot: __Initial, Choose the type of coffee and Order Complete.__

Based on the message received from the user, we need to define the next state of the bot along with the response.

In [ ]:
# Define the INIT state
INIT=0
# Define the CHOOSE_COFFEE state
CHOOSE_COFFEE=1
# Define the ORDERED state
ORDERED=2

# Define the policy rules
policy = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
}

# Create the list of messages
messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

# Call send_message() for each message
state = INIT
for message in messages:    
    state = send_message(policy, state, message)

USER : I'd like to become a professional dancer

BOT : I'm sorry - I'm not sure how to help you

USER : well then I'd like to order some coffee

BOT : ok, Colombian or Kenyan?

USER : my favourite animal is a zebra

BOT : I'm sorry - would you like Colombian or Kenyan?

USER : kenyan

BOT : perfect, the beans are on their way!

### 3.1. Including intent with the state
The code snippet below involves more number of intents like 'ask_explanation' for each of the states.
The bot should respond in a different manner in each of the state based on the intent of the message as shown below.

In [ ]:
# Define the states
INIT=0 
CHOOSE_COFFEE=1
ORDERED=2

# Define the policy rules dictionary
policy_rules = {
    (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Columbian or Kenyan?"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    for msg in messages:
        state = send_message(state, msg)

# Send the messages
send_messages([
    "what can you do for me?",
    "well then I'd like to order some coffee",
    "what do you mean by that?",
    "kenyan"
])

USER : what can you do for me?

BOT : I'm a bot to help you order coffee beans

USER : well then I'd like to order some coffee

BOT : ok, Columbian or Kenyan?

USER : what do you mean by that?

BOT : We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost \$6. The Brazilian beans make a nutty coffee and cost $5.

USER : kenyan

BOT : perfect, the beans are on their way!

#### 3.1.1. Dealing with rejection

This section will help you address scenarios when the user does not like a suggestion made by the bot and responds to the bot with a rejection.

In such a case, you don't want the bot to look silly and respond with the same suggestion over and over again.

In [ ]:
# Define respond()
def respond(message, params, prev_suggestions, excluded):
    # Interpret the message
    parse_data = interpret(message)
    # Extract the intent
    intent = parse_data["intent"]["name"]
    # Extract the entities
    entities = parse_data["entities"]
    # Add the suggestion to the excluded list if intent is "deny"
    if intent == "deny":
        excluded.extend(prev_suggestions) 
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find matching hotels
    results = [
        r for r in find_hotels(params, excluded) 
        if r[0] not in excluded
    ]
    # Extract the suggestions
    names = [r[0] for r in results]
    n = min(len(results), 3)
    suggestions = names[:2]
    return responses[n].format(*names), params, suggestions, excluded

# Initialize the empty dictionary and lists
params, suggestions, excluded = {}, [], []

# Send the messages
for message in ["I want a mid range hotel", "no that doesn't work for me"]:
    print("USER: {}".format(message))
    response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
    print("BOT: {}".format(response))


USER: I want a mid range hotel

BOT: Hotel for Dogs is one option, but I know others too :)

USER: no that doesn't work for me

BOT: Grand Hotel is one option, but I know others too :)

#### 3.1.2. Asking Questions and Queuing Answers

This section involves improving the user experience of the bot by asking simple yes or no questions.

A good way of handling these follow-ups is to define pending actions which get executed as soon as the user says 'yes', else wipe the action if the user says 'no'.

In [ ]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

# Define send_message()
def send_message(pending, message):
    print("USER : {}".format(message))
    action, pending_action = policy(interpret(message))
    if action == "do_pending" and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending = None
    for msg in messages:
        pending = send_message(pending,msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

USER : I'd like to order some coffee

BOT : Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?

USER : ok yes please

BOT : Alright, I've ordered that for you!

In [ ]:
# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Columbian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg )

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "kenyan"
])


USER : I'd like to order some coffee

BOT : you'll have to log in first, what's your phone number?

USER : 555-12345

BOT : perfect, welcome back!

BOT : would you like Columbian or Kenyan?

USER : kenyan

BOT : perfect, the beans are on their way!

BOT : would you like Columbian or Kenyan?

## 4. Implementing all the rules to make a prototype

The below section involves putting all the things together and then observing the bot assist a customer in ordering coffee beans.

In [ ]:
eliza_rules = {'I want (.*)': ['What would it mean if you got {0}',
  'Why do you want {0}',
  "What's stopping you from getting {0}"],
 'do you remember (.*)': ['Did you think I would forget {0}',
  "Why haven't you been able to forget {0}",
  'What about {0}',
  'Yes .. and?'],
 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'],
 'if (.*)': ["Do you really think it's likely that {0}",
  'Do you wish that {0}',
  'What do you think about {0}',
  'Really--if {0}']}

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern, responses in rules.items():
        # Create a match object
        match = re.search(pattern,message)
        if match is not None:
            # Choose a random response
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

def replace_pronouns(message):
    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me','you',message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my','your',message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your','my',message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you','me',message)
    return message

# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(eliza_rules, message)
    # Return none is response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(phrase)
        # Calculate the response
        response = response.format(phrase)
    return response

### Include missing functions


# Define send_message()
def send_message(state, pending, message):
    print("USER : {}".format(message))
    response = chitchat_response(message)
    if response is not None:
        print("BOT : {}".format(response))
        return state, None
    
    # Calculate the new_state, response, and pending_state
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "do you remember when I ordered 1000 kilos by accident?",
    "kenyan"
]) 

USER : I'd like to order some coffee

BOT : you'll have to log in first, what's your phone number?

USER : 555-12345

BOT : perfect, welcome back!

BOT : would you like Columbian or Kenyan?

USER : do you remember when I ordered 1000 kilos by accident?

BOT : Why haven't you been able to forget when you ordered 1000 kyoulos by accyoudent?

USER : kenyan

BOT : perfect, the beans are on their way!

## 5. Setting up a Facebook messenger bot

Facebook being a popular option, the next step would be to integrate the sections of code above with Facebook messenger and host a bot on the social media platform using ngrok and Flask.

The messenger bot was setup by following instructions shared on the following URL:
https://tutorials.botsfloor.com/using-ngrok-for-testing-your-messenger-bot-22a84f8185fb